In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn import model_selection, preprocessing, metrics
import scipy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#train = pd.read_hdf('/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/dataset/train_cite_inputs.h5')
#test = pd.read_hdf('/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/dataset/test_cite_inputs.h5')

In [4]:
train = scipy.sparse.load_npz("/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/dataset/train_cite_inputs_values.sparse.npz")
test = scipy.sparse.load_npz('/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/dataset/test_cite_inputs_values.sparse.npz')

In [5]:
train

<70988x22050 sparse matrix of type '<class 'numpy.float32'>'
	with 344303478 stored elements in Compressed Sparse Row format>

In [7]:
train.shape

(70988, 22050)

In [6]:
train['target'] = 0
test['target'] = 1

IndexError: ignored

In [5]:
train_test = pd.concat([train, test], axis =0)

In [6]:
train, test = model_selection.train_test_split(train_test, test_size=0.33, random_state=42, shuffle=True)

In [7]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.2,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 44,
         "metric": 'auc',
         "verbosity": -1}

In [8]:
metadata_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/dataset/metadata.csv', index_col='cell_id')

In [9]:
metadata_df.head()

,day,donor,cell_type,technology
cell_id,,,,
c2150f55becb,2,27678,HSC,citeseq
65b7edf8a4da,2,27678,HSC,citeseq
c1b26cb1057b,2,27678,EryP,citeseq
917168fa6f83,2,27678,NeuP,citeseq
2b29feeca86d,2,27678,EryP,citeseq


In [10]:
metadata_df.shape

(281528, 4)

In [11]:
metadata_df = metadata_df[metadata_df.technology=="citeseq"]
metadata_df.shape

(119651, 4)

In [12]:
cell_index =np.load("/content/drive/MyDrive/Colab Notebooks/kaggle/MSCI/dataset/train_cite_targets_idxcol.npz",
                   allow_pickle=True)["index"]
meta = metadata_df.reindex(cell_index)

In [13]:
meta.head()

,day,donor,cell_type,technology
cell_id,,,,
45006fe3e4c8,2,32606,HSC,citeseq
d02759a80ba2,2,32606,HSC,citeseq
c016c6b0efa5,2,32606,EryP,citeseq
ba7f733a4f75,2,32606,NeuP,citeseq
fbcf2443ffb2,2,32606,EryP,citeseq


In [8]:
num_round = 100

In [9]:
train_y = train['target'].values
test_y = test['target'].values
train = lgb.Dataset(train, label=train_y)
test = lgb.Dataset(test, label=test_y)

In [ ]:
clf = lgb.train(param, train, num_round, valid_sets = [train, test], verbose_eval=50, early_stopping_rounds = 50)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(),features)), columns=['Value','Feature'])

plt.figure(figsize=(20, 300))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).head(500))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
#plt.savefig('lgbm_importances-01.png')

In [ ]:
ax = lgb.plot_metric(model.evals_result_, metric='auc')
plt.show()